# Fibre preserving symmetries in the phase-plane
*Date*: 2022-06-07,<br>
*Written by:* Johannes Borgqvist.<br>
Now, we study a general time-invariant two component system of ODEs:
\begin{align}
    \dfrac{\mathrm{d}u}{\mathrm{d}t}&=\omega_1(u,v),\\
    \dfrac{\mathrm{d}v}{\mathrm{d}t}&=\omega_2(u,v).\\
\end{align}
and we are interested in this type of infinitesimal generator of the Lie group:
\begin{equation}
X = \eta_1(u,v)\partial_u+\eta_2(u,v)\partial_v.
  \label{eq:generator_phase_plane}
\end{equation}
It turns out that for this type of generators there is a single linearised symmetry condition that corresponds to solving the following PDE:
\begin{equation}
 \omega_1^2 \dfrac{\partial\eta_2}{\partial u}+\omega_1\omega_2\left(\dfrac{\partial\eta_2}{\partial v}-\dfrac{\partial\eta_1}{\partial u}\right)-\omega_2^2\dfrac{\partial\eta_1}{\partial v}=\left(\dfrac{\partial\omega_2}{\partial u}\omega_1-\omega_2\dfrac{\partial\omega_1}{\partial u}\right)\eta_1+\left(\dfrac{\partial\omega_2}{\partial v}\omega_1-\omega_2\dfrac{\partial\omega_1}{\partial v}\right)\eta_2.
\label{eq:lin_sym_fibre}
\end{equation}
Now, we would like to see if we can find any solutions for $\eta_1(u,v)$ and $\eta_2(u,v)$ using polynomial ans\"atze, and we will try to solve these equations using *SymPy*. <br>

In particular, we will look at the Lotka-Volterra (LV) model, the Belusov-Zhabotinskii (BZ) model and the Brusselator model. They all have polynomial reaction terms $\omega_1(u,v)$ and $\omega_2(u,v)$.

# Import libraries

In [1]:
# Import sympy
from sympy import *
# Translate a string to symbolic expression
from sympy.parsing.sympy_parser import parse_expr
# Finding monomials
from sympy.polys.monomials import itermonomials, monomial_count
# Ordering monomials 
from sympy.polys.orderings import monomial_key
# For printing to a file
import sys
# Import mathematical function
import math as m
# To extract all subsets of the coefficient vector
#from itertools import chain, combinations
import itertools
# Import numpy as well
import numpy as np

# Functions


In [2]:
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 1: "create_tangent_ansatze"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# The function takes two inputs:
#1. The states in the list states, 
#2. The degree of the polynomial ("degree_polynomial").
# It returns three outputs:
# 1.The unknown coefficients in the tangential ansatze,
# 2. A list of all tangents ("eta_list"),
# 3. The list of all monomials.
# The function uses various functionalities in sympy and the polynomials are generated using the built in functions in the polys.monomial part of the sympy library. To automate the generation of the sympy functions and symbol, the "exec" command is used at multiple places in the function.
def create_tangential_ansatze(states,degree_polynomial):
    #----------------------------------------------------------------------------------
    # INITIALISATION: ALLOCATE MEMORY FOR OUR OUTPUT
    #----------------------------------------------------------------------------------
    # Calculate the number of variables and number of states
    num_of_states = len(states)
    # Allocate our to lists which will be the output:
    eta_list = [] # The function with the tangents        
    c = [] # The list of the coefficients
    #----------------------------------------------------------------------------------
    # STEP 1: GENERATE POLYNOMIALS IN THE TEMPORARY VARIABLES
    #----------------------------------------------------------------------------------
    # Generate all monomials for our polynomials
    M = list(itermonomials(states, degree_polynomial))
    # Sort the list
    M = sorted(M, key=monomial_key('lex', states))
    # Calculate the number of terms in each of the polynomials
    num_of_monomials = monomial_count(num_of_states,degree_polynomial)
    #----------------------------------------------------------------------------------
    # STEP 2: DEFINE THE POLYNOMIAL ANSATZE
    #----------------------------------------------------------------------------------
    # Allocate our common polynomial
    P_0 = symbols('P_0', cls=Function)
    # Loop over the states and allocate our polynomial
    for state_index in range(num_of_states):
        # Initialise it to zero
        P_0 = 0    
        # Loop over the number of terms
        for monomial_index,monomial in enumerate(M):
            # Define the current index of our coefficient
            index = monomial_index + state_index*num_of_monomials
            # Allocate a coefficient
            exec("c_%d = symbols(\'c_%d\') "%(index,index)) 
            # Add this coefficient to the set of unknowns
            exec("c.append(c_%d)"%(index))
            # Add this coefficent to our polynomial at hand
            P_0+=c[-1]*M[monomial_index]
        # When we are done we add P_0 to our list of tangential ansatze
        eta_list.append(P_0)
    # Return the output    
    return c, eta_list, M
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 2: "lin_sym_phase_plane"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# This function sets up the linearised symmetry condition for the phase plane. 
# It takes three inputs:
# 1. The states,
# 2. The reaction terms,
# 3. The tangents.
# It returns three outputs:
# 1. The linearised symmetry condition in a variable called lin_sym,
# 2. The system of equation stemming from the monomials being linearly independent,
# 3. The monomials that each equation stems from.
def  lin_sym_phase_plane(states,omega,eta):
    # Allocate memory for our outputs
    lin_sym = [] # The linearised symmetry condition
    eq_sys = [] # The system of equations
    monomial_sys = [] # The monomials each equation stems from
    #----------------------------------------------------------------
    # STEP 1: SETUP THE LINEARISED SYMMETRY CONDITION
    #----------------------------------------------------------------
    # The linearised symmetry condition is hard coded
    temp_eq = 0
    # The LHS
    temp_eq += omega[0]**2*Derivative(eta[1],states[0]).doit()
    temp_eq += omega[0]*omega[1]*(Derivative(eta[1],states[1]).doit()-Derivative(eta[0],states[0]).doit())
    temp_eq += -omega[1]**2*Derivative(eta[0],states[1]).doit()
    # The RHS
    temp_eq += -(Derivative(omega[1],states[0]).doit()*omega[0]- omega[1]*Derivative(omega[0],states[0]).doit())*eta[0]
    temp_eq += -(Derivative(omega[1],states[1]).doit()*omega[0]- omega[1]*Derivative(omega[0],states[1]).doit())*eta[1]
    # Append our lovely equation
    lin_sym.append(expand(temp_eq))
    #----------------------------------------------------------------
    # STEP 2: EXTRACT THE EQUATIONS STEMMING FROM THE LIN SYM
    #----------------------------------------------------------------    
    # Generate all monomials of a high defree
    M = list(itermonomials(states, 20))
    # Sort the list
    M = sorted(M, key=monomial_key('lex', states))
    # Loop through the monomials, and extract all coefficients that are non-zero
    for m_index,monomial in enumerate(M):
        # Calculate the coefficient at hand
        temp_eq = lin_sym[0].coeff(monomial)
        # Set all higher order monomials to zero
        for state_index,state in enumerate(states):
            temp_eq = temp_eq.subs(state,0)
        # If we have a non-zero equation we append it to our list of equations
        if temp_eq != 0:
            eq_sys.append(temp_eq)
            monomial_sys.append(monomial)
    # Return the output 
    return lin_sym,eq_sys,monomial_sys
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 3: "extract_generators"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# This function sets up the linearised symmetry condition for the phase plane. 
# It takes four inputs:
# 1. c the coefficients in the initial ansatze,
# 2. LHS being the coefficients we have solved for,
# 3. RHS being the value of these coefficients in terms of other coefficients,
# 4. eta being the list of the initial tangents.
# It returns a single output being the list of all generators as a tuple of the two 
# infinitesimals namely (eta_1,eta_2).
def extract_generators(c,LHS,RHS,eta):
    # Allocate memory of our output
    generator_list = []    
    #----------------------------------------------------------------
    # STEP 1: FIND THE CONSTANTS THAT APPEAR IN THE FINAL SOLUTION
    #----------------------------------------------------------------
    # See which constants that exists among the constants
    constants_final = []
    # Loop over all algebraic expressions
    for RHS_index,RHS_temp in enumerate(RHS):
            # Loop over all candidate constants
            for c_index,c_temp in enumerate(c):
                # See if the constant exists in the final solution
                if RHS_temp.coeff(c_temp) != 0:
                    # Add the constant
                    constants_final.append(c_temp)
    # Only save the unique ones
    constants_final = list(set(constants_final))
    #----------------------------------------------------------------
    # STEP 2: SUBSTITUTE CONSTANTS INTO THE TANGENTS
    #----------------------------------------------------------------
    # Allocate memory for the final tangents
    eta_final = eta
    # Loop through the tangents and substitute the values for the coefficients
    for eta_index,eta_temp in enumerate(eta_final):
    # Loop through the coefficients as well
        for c_index,LHS_temp in enumerate(LHS):
            eta_final[eta_index] = eta_final[eta_index].subs(LHS_temp,RHS[c_index])
        # Simplify in the end to clean it up
        eta_final[eta_index] = expand(eta_final[eta_index])
    #----------------------------------------------------------------
    # STEP 3: EXTRACT GENERATORS
    #----------------------------------------------------------------    
    # ALlocate memory for the trivial generator
    trivial_generator = eta_final
    trivial_generator_returned = []
    # Loop over all our coefficients and save our generators
    for constant_index,constant in enumerate(constants_final):
        # Append the current generator
        generator_list.append((factor(eta_final[0].coeff(constant)),factor(eta_final[1].coeff(constant))))
        # Substract the current generator from the trivial generator
        trivial_generator[0] = expand(trivial_generator[0] - eta_final[0].coeff(constant)*constant)
        trivial_generator[1] = expand(trivial_generator[1] - eta_final[1].coeff(constant)*constant)
    # Lastly, append the trivial generator as well
    if simplify(trivial_generator[0]) != 0 and simplify(trivial_generator[1])!= 0:
        trivial_generator_returned.append((simplify(trivial_generator[0]),simplify(trivial_generator[1])))
    # Return our generators
    return generator_list, trivial_generator_returned

# The LV-model

Now, we are considering the Lotka-Volterra model:

\begin{equation}
  \begin{split}
    \dfrac{\mathrm{d}u}{\mathrm{d}\tau}&=u(1-v),\\
    \dfrac{\mathrm{d}v}{\mathrm{d}\tau}&=\alpha v(u-1).\\    
    \end{split}
  \label{eq:LV}
\end{equation}

### Set up the reaction terms

In [3]:
# Dependent variables
u,v = symbols('u v')
# Define our parameter a in the LV model
a = symbols('a')
# Define the states
states_LV = [u, v]
# Define our reaction terms
omega_LV = [u*(1-v), a*v*(u-1)]

### Generate tangential ansatze

In [4]:
c_LV, eta_LV, M_LV = create_tangential_ansatze(states_LV,5)
print("The monomials:")
print(latex(Matrix(len(M_LV),1,M_LV),mode='equation').replace("\\begin{equation}","\\begin{equation}\n\\mathbf{M}=").replace("\\end{equation}",".\n\\end{equation}"))
print("The unknown coefficients:")
print(latex(Matrix(len(c_LV),1,c_LV),mode='equation').replace("\\begin{equation}","\\begin{equation}\n\\mathbf{c}=").replace("\\end{equation}",".\n\\end{equation}"))
print("The tangential ansatze:")
print("\\begin{align*}")
for state_index,state in enumerate(states_LV):
    if state_index == len(states_LV)-1:
        print("\\eta_%d(%s,%s)&=%s.\\\\"%(state_index+1,latex(states_LV[0]),latex(states_LV[1]),latex(eta_LV[state_index])))
    else:
        print("\\eta_%d(%s,%s)&=%s,\\\\"%(state_index+1,latex(states_LV[0]),latex(states_LV[1]),latex(eta_LV[state_index])))
print("\\end{align*}")

The monomials:
\begin{equation}
\mathbf{M}=\left[\begin{matrix}1\\v\\v^{2}\\v^{3}\\v^{4}\\v^{5}\\u\\u v\\u v^{2}\\u v^{3}\\u v^{4}\\u^{2}\\u^{2} v\\u^{2} v^{2}\\u^{2} v^{3}\\u^{3}\\u^{3} v\\u^{3} v^{2}\\u^{4}\\u^{4} v\\u^{5}\end{matrix}\right].
\end{equation}
The unknown coefficients:
\begin{equation}
\mathbf{c}=\left[\begin{matrix}c_{0}\\c_{1}\\c_{2}\\c_{3}\\c_{4}\\c_{5}\\c_{6}\\c_{7}\\c_{8}\\c_{9}\\c_{10}\\c_{11}\\c_{12}\\c_{13}\\c_{14}\\c_{15}\\c_{16}\\c_{17}\\c_{18}\\c_{19}\\c_{20}\\c_{21}\\c_{22}\\c_{23}\\c_{24}\\c_{25}\\c_{26}\\c_{27}\\c_{28}\\c_{29}\\c_{30}\\c_{31}\\c_{32}\\c_{33}\\c_{34}\\c_{35}\\c_{36}\\c_{37}\\c_{38}\\c_{39}\\c_{40}\\c_{41}\end{matrix}\right].
\end{equation}
The tangential ansatze:
\begin{align*}
\eta_1(u,v)&=c_{0} + c_{1} v + c_{10} u v^{4} + c_{11} u^{2} + c_{12} u^{2} v + c_{13} u^{2} v^{2} + c_{14} u^{2} v^{3} + c_{15} u^{3} + c_{16} u^{3} v + c_{17} u^{3} v^{2} + c_{18} u^{4} + c_{19} u^{4} v + c_{2} v^{2} + c_{20} u^{5} + c_{3} v^{3} + c_{4} v^{4} + c_{

The monomials:
\begin{equation}
\mathbf{M}=\left[\begin{matrix}1\\v\\v^{2}\\v^{3}\\v^{4}\\v^{5}\\u\\u v\\u v^{2}\\u v^{3}\\u v^{4}\\u^{2}\\u^{2} v\\u^{2} v^{2}\\u^{2} v^{3}\\u^{3}\\u^{3} v\\u^{3} v^{2}\\u^{4}\\u^{4} v\\u^{5}\end{matrix}\right].
\end{equation}
The unknown coefficients:
\begin{equation}
\mathbf{c}=\left[\begin{matrix}c_{0}\\c_{1}\\c_{2}\\c_{3}\\c_{4}\\c_{5}\\c_{6}\\c_{7}\\c_{8}\\c_{9}\\c_{10}\\c_{11}\\c_{12}\\c_{13}\\c_{14}\\c_{15}\\c_{16}\\c_{17}\\c_{18}\\c_{19}\\c_{20}\\c_{21}\\c_{22}\\c_{23}\\c_{24}\\c_{25}\\c_{26}\\c_{27}\\c_{28}\\c_{29}\\c_{30}\\c_{31}\\c_{32}\\c_{33}\\c_{34}\\c_{35}\\c_{36}\\c_{37}\\c_{38}\\c_{39}\\c_{40}\\c_{41}\end{matrix}\right].
\end{equation}
The tangential ansatze:
\begin{align*}
\eta_1(u,v)&=c_{0} + c_{1} v + c_{10} u v^{4} + c_{11} u^{2} + c_{12} u^{2} v + c_{13} u^{2} v^{2} + c_{14} u^{2} v^{3} + c_{15} u^{3} + c_{16} u^{3} v + c_{17} u^{3} v^{2} + c_{18} u^{4} + c_{19} u^{4} v + c_{2} v^{2} + c_{20} u^{5} + c_{3} v^{3} + c_{4} v^{4} + c_{5} v^{5} + c_{6} u + c_{7} u v + c_{8} u v^{2} + c_{9} u v^{3},\\
\eta_2(u,v)&=c_{21} + c_{22} v + c_{23} v^{2} + c_{24} v^{3} + c_{25} v^{4} + c_{26} v^{5} + c_{27} u + c_{28} u v + c_{29} u v^{2} + c_{30} u v^{3} + c_{31} u v^{4} + c_{32} u^{2} + c_{33} u^{2} v + c_{34} u^{2} v^{2} + c_{35} u^{2} v^{3} + c_{36} u^{3} + c_{37} u^{3} v + c_{38} u^{3} v^{2} + c_{39} u^{4} + c_{40} u^{4} v + c_{41} u^{5}.\\
\end{align*}



### Setup the linearised symmetry condition

In [5]:
# Plug in the tangential ansatze and calculate the resulting equations
lin_sym_LV,eq_sys_LV,monomials_LV = lin_sym_phase_plane(states_LV,omega_LV,eta_LV)
# Print these equations
print("The equations stemming from the linearised symmetry conditions:")
print("\\begin{align*}")
for eq_index,eq_temp in enumerate(eq_sys_LV):
    if eq_index == len(eq_sys_LV)-1:
        print("%s:&%s=0.\\\\"%(latex(monomials_LV[eq_index]),latex(eq_temp)))
    else:
        print("%s:&%s=0,\\\\"%(latex(monomials_LV[eq_index]),latex(eq_temp)))
print("\\end{align*}")

The equations stemming from the linearised symmetry conditions:
\begin{align*}
v:&- a c_{0}=0,\\
v^{2}:&- a^{2} c_{1} + a c_{0} - a c_{1}=0,\\
v^{3}:&- 2 a^{2} c_{2} + a c_{1} - a c_{2}=0,\\
v^{4}:&- 3 a^{2} c_{3} + a c_{2} - a c_{3}=0,\\
v^{5}:&- 4 a^{2} c_{4} + a c_{3} - a c_{4}=0,\\
v^{6}:&- 5 a^{2} c_{5} + a c_{4} - a c_{5}=0,\\
v^{7}:&a c_{5}=0,\\
u:&a c_{21}=0,\\
u v^{2}:&2 a^{2} c_{1} - a^{2} c_{7} + a c_{22} - a c_{23}=0,\\
u v^{3}:&4 a^{2} c_{2} - 2 a^{2} c_{8} + 2 a c_{23} - 2 a c_{24}=0,\\
u v^{4}:&6 a^{2} c_{3} - 3 a^{2} c_{9} + 3 a c_{24} - 3 a c_{25}=0,\\
u v^{5}:&- 4 a^{2} c_{10} + 8 a^{2} c_{4} + 4 a c_{25} - 4 a c_{26}=0,\\
u v^{6}:&10 a^{2} c_{5} + 5 a c_{26}=0,\\
u^{2}:&- a c_{21} + a c_{27} + c_{27}=0,\\
u^{2} v:&a c_{11} - a c_{6} - 2 c_{27} + c_{28}=0,\\
u^{2} v^{2}:&- a^{2} c_{1} - a^{2} c_{12} + 2 a^{2} c_{7} - a c_{11} + a c_{12} - a c_{22} + a c_{23} + a c_{28} - a c_{29} + a c_{6} - a c_{7} + c_{27} - 2 c_{28} + c_{29}=0,\\
u^{2} v^{3}:&- 2 a^{2} c_{13} - 2 a

The equations stemming from the linearised symmetry conditions:
\begin{align*}
v:&- a c_{0}=0,\\
v^{2}:&a c_{0} - a c_{1}=0,\\
v^{3}:&a c_{1} - a c_{2}=0,\\
v^{4}:&a c_{2} - a c_{3}=0,\\
v^{5}:&a c_{3} - a c_{4}=0,\\
v^{6}:&a c_{4} - a c_{5}=0,\\
v^{7}:&a c_{5}=0,\\
u:&a c_{21}=0,\\
u v^{2}:&a^{2} + a c_{22} - a c_{23}=0,\\
u v^{3}:&2 a c_{23} - 2 a c_{24}=0,\\
u v^{4}:&3 a c_{24} - 3 a c_{25}=0,\\
u v^{5}:&4 a c_{25} - 4 a c_{26}=0,\\
u v^{6}:&5 a c_{26}=0,\\
u^{2}:&- a c_{21} + a c_{27}=0,\\
u^{2} v:&a c_{11} - a c_{6} + a=0,\\
u^{2} v^{2}:&- 2 a^{2} - a c_{11} + a c_{12} - a c_{22} + a c_{23} + a c_{28} - a c_{29} + a c_{6} - a c_{7} - 2 a=0,\\
u^{2} v^{3}:&- a c_{12} + a c_{13} - 2 a c_{23} + 2 a c_{24} + 2 a c_{29} - 2 a c_{30} + a c_{7} - a c_{8} + a=0,\\
u^{2} v^{4}:&- a c_{13} + a c_{14} - 3 a c_{24} + 3 a c_{25} + 3 a c_{30} - 3 a c_{31} + a c_{8} - a c_{9}=0,\\
u^{2} v^{5}:&- a c_{10} - a c_{14} - 4 a c_{25} + 4 a c_{26} + 4 a c_{31} + a c_{9}=0,\\
u^{2} v^{6}:&a c_{10} - 5 a c_{26}=0,\\
u^{3}:&- a c_{27} + a c_{32}=0,\\
u^{3} v:&- 2 a c_{11} + 2 a c_{15}=0,\\
u^{3} v^{2}:&a^{2} + 2 a c_{11} - 2 a c_{12} - 2 a c_{15} + 2 a c_{16} - a c_{28} + a c_{29} + a c_{33} - a c_{34}=0,\\
u^{3} v^{3}:&2 a c_{12} - 2 a c_{13} - 2 a c_{16} + 2 a c_{17} - 2 a c_{29} + 2 a c_{30} + 2 a c_{34} - 2 a c_{35}=0,\\
u^{3} v^{4}:&2 a c_{13} - 2 a c_{14} - 2 a c_{17} - 3 a c_{30} + 3 a c_{31} + 3 a c_{35}=0,\\
u^{3} v^{5}:&2 a c_{14} - 4 a c_{31}=0,\\
u^{4}:&- a c_{32} + a c_{36}=0,\\
u^{4} v:&- 3 a c_{15} + 3 a c_{18}=0,\\
u^{4} v^{2}:&3 a c_{15} - 3 a c_{16} - 3 a c_{18} + 3 a c_{19} - a c_{33} + a c_{34} + a c_{37} - a c_{38}=0,\\
u^{4} v^{3}:&3 a c_{16} - 3 a c_{17} - 3 a c_{19} - 2 a c_{34} + 2 a c_{35} + 2 a c_{38}=0,\\
u^{4} v^{4}:&3 a c_{17} - 3 a c_{35}=0,\\
u^{5}:&- a c_{36} + a c_{39}=0,\\
u^{5} v:&- 4 a c_{18} + 4 a c_{20}=0,\\
u^{5} v^{2}:&4 a c_{18} - 4 a c_{19} - 4 a c_{20} - a c_{37} + a c_{38} + a c_{40}=0,\\
u^{5} v^{3}:&4 a c_{19} - 2 a c_{38}=0,\\
u^{6}:&- a c_{39} + a c_{41}=0,\\
u^{6} v:&- 5 a c_{20}=0,\\
u^{6} v^{2}:&5 a c_{20} - a c_{40}=0,\\
u^{7}:&- a c_{41}=0.\\
\end{align*}


### Solve the final system for the unknown coefficients

In [6]:
# Solve the system of equations for the unknown coefficients
solutions_LV = solve(eq_sys_LV,c_LV,set=True)
# Extract the LHS and RHS 
LHS_LV = solutions_LV[0]
RHS_LV = list(list(solutions_LV[1])[0])
solution_str = "\\begin{equation}\n" + latex(Matrix(len(LHS_LV),1,LHS_LV)) + "=" + latex(Matrix(len(RHS_LV),1,RHS_LV)) + "\n\\end{equation}"
print("The solution is:\n%s\n"%(solution_str))
# Calculate and extract all generators
generators_LV,trivial_generator = extract_generators(c_LV,LHS_LV,RHS_LV,eta_LV)
print("The calculated infinitesimal generators of the Lie group:")
print("\\begin{align*}")
for gen_index in range(len(generators_LV)):
    if gen_index <len(generators_LV)-1:
        print("X_{%d}&=%s\\partial_u+%s\\partial_v,\\\\"%(gen_index+1,latex(generators_LV[gen_index][0]),latex(generators_LV[gen_index][1])))
    else:
        print("X_{%d}&=%s\\partial_u+%s\\partial_v.\\\\"%(gen_index+1,latex(generators_LV[gen_index][0]),latex(generators_LV[gen_index][1])))
print("\\end{align*}")
if len(trivial_generator)>0:
    print("The trivial generator:")
    trivial_str = "\\begin{equation}\nX_{0}="+latex(trivial_generator[0][0])+"\\partial_u+"+latex(trivial_generator[0][1])+"\\partial_v.\n\\end{equation}"
    print("%s"%(trivial_str))
print("The reaction terms:")
print("\\begin{align*}\n\omega_1(u,v)&=%s,\\\\\n\omega_2(u,v)&=%s.\\\\\n\\end{align*}"%(latex(omega_LV[0]),latex(omega_LV[1])))

The solution is:
\begin{equation}
\left[\begin{matrix}c_{0}\\c_{1}\\c_{10}\\c_{11}\\c_{12}\\c_{13}\\c_{14}\\c_{15}\\c_{16}\\c_{17}\\c_{18}\\c_{19}\\c_{2}\\c_{20}\\c_{21}\\c_{22}\\c_{23}\\c_{24}\\c_{25}\\c_{26}\\c_{27}\\c_{3}\\c_{32}\\c_{36}\\c_{39}\\c_{4}\\c_{41}\\c_{5}\\c_{6}\\c_{7}\\c_{8}\\c_{9}\end{matrix}\right]=\left[\begin{matrix}0\\0\\- \frac{c_{31}}{a}\\\frac{c_{33}}{a} + \frac{c_{37}}{a} + \frac{c_{40}}{a}\\- \frac{c_{33}}{a} + \frac{c_{34}}{a} - \frac{c_{37}}{a} + \frac{c_{38}}{a} - \frac{c_{40}}{a}\\- \frac{c_{34}}{a} + \frac{c_{35}}{a} - \frac{c_{38}}{a}\\- \frac{c_{35}}{a}\\\frac{c_{37}}{a} + \frac{c_{40}}{a}\\- \frac{c_{37}}{a} + \frac{c_{38}}{a} - \frac{c_{40}}{a}\\- \frac{c_{38}}{a}\\\frac{c_{40}}{a}\\- \frac{c_{40}}{a}\\0\\0\\0\\- c_{28} - c_{33} - c_{37} - c_{40}\\- c_{29} - c_{34} - c_{38}\\- c_{30} - c_{35}\\- c_{31}\\0\\0\\0\\0\\0\\0\\0\\0\\0\\\frac{c_{28}}{a} + \frac{c_{33}}{a} + \frac{c_{37}}{a} + \frac{c_{40}}{a}\\- \frac{c_{28}}{a} + \frac{c_{29}}{a} - \frac{c_

The solution is:
\begin{equation}
\left[\begin{matrix}c_{0}\\c_{1}\\c_{10}\\c_{11}\\c_{12}\\c_{13}\\c_{14}\\c_{15}\\c_{16}\\c_{17}\\c_{18}\\c_{19}\\c_{2}\\c_{20}\\c_{21}\\c_{22}\\c_{23}\\c_{24}\\c_{25}\\c_{26}\\c_{27}\\c_{3}\\c_{32}\\c_{36}\\c_{39}\\c_{4}\\c_{41}\\c_{5}\\c_{6}\\c_{7}\\c_{8}\\c_{9}\end{matrix}\right]=\left[\begin{matrix}0\\0\\- \frac{c_{31}}{a}\\\frac{c_{33}}{a} + \frac{c_{37}}{a} + \frac{c_{40}}{a}\\- \frac{c_{33}}{a} + \frac{c_{34}}{a} - \frac{c_{37}}{a} + \frac{c_{38}}{a} - \frac{c_{40}}{a}\\- \frac{c_{34}}{a} + \frac{c_{35}}{a} - \frac{c_{38}}{a}\\- \frac{c_{35}}{a}\\\frac{c_{37}}{a} + \frac{c_{40}}{a}\\- \frac{c_{37}}{a} + \frac{c_{38}}{a} - \frac{c_{40}}{a}\\- \frac{c_{38}}{a}\\\frac{c_{40}}{a}\\- \frac{c_{40}}{a}\\0\\0\\0\\- c_{28} - c_{33} - c_{37} - c_{40}\\- c_{29} - c_{34} - c_{38}\\- c_{30} - c_{35}\\- c_{31}\\0\\0\\0\\0\\0\\0\\0\\0\\0\\\frac{c_{28}}{a} + \frac{c_{33}}{a} + \frac{c_{37}}{a} + \frac{c_{40}}{a}\\- \frac{c_{28}}{a} + \frac{c_{29}}{a} - \frac{c_{33}}{a} + \frac{c_{34}}{a} - \frac{c_{37}}{a} + \frac{c_{38}}{a} - \frac{c_{40}}{a}\\- \frac{c_{29}}{a} + \frac{c_{30}}{a} - \frac{c_{34}}{a} + \frac{c_{35}}{a} - \frac{c_{38}}{a}\\- \frac{c_{30}}{a} + \frac{c_{31}}{a} - \frac{c_{35}}{a}\end{matrix}\right]
\end{equation}

The calculated infinitesimal generators of the Lie group:
\begin{align*}
X_{1}&=- \frac{u v^{2} \left(u + 1\right) \left(v - 1\right)}{a}\partial_u+v^{3} \left(u - 1\right) \left(u + 1\right)\partial_v,\\
X_{2}&=- \frac{u v \left(v - 1\right) \left(u^{2} + u + 1\right)}{a}\partial_u+v^{2} \left(u - 1\right) \left(u^{2} + u + 1\right)\partial_v,\\
X_{3}&=- \frac{u \left(v - 1\right) \left(u^{2} + u + 1\right)}{a}\partial_u+v \left(u - 1\right) \left(u^{2} + u + 1\right)\partial_v,\\
X_{4}&=- \frac{u \left(v - 1\right)}{a}\partial_u+v \left(u - 1\right)\partial_v,\\
X_{5}&=- \frac{u v^{2} \left(v - 1\right)}{a}\partial_u+v^{3} \left(u - 1\right)\partial_v,\\
X_{6}&=- \frac{u v^{3} \left(v - 1\right)}{a}\partial_u+v^{4} \left(u - 1\right)\partial_v,\\
X_{7}&=- \frac{u v \left(v - 1\right)}{a}\partial_u+v^{2} \left(u - 1\right)\partial_v,\\
X_{8}&=- \frac{u \left(u + 1\right) \left(u^{2} + 1\right) \left(v - 1\right)}{a}\partial_u+v \left(u - 1\right) \left(u + 1\right) \left(u^{2} + 1\right)\partial_v,\\
X_{9}&=- \frac{u \left(u + 1\right) \left(v - 1\right)}{a}\partial_u+v \left(u - 1\right) \left(u + 1\right)\partial_v,\\
X_{10}&=- \frac{u v \left(u + 1\right) \left(v - 1\right)}{a}\partial_u+v^{2} \left(u - 1\right) \left(u + 1\right)\partial_v.\\
\end{align*}
The reaction terms:
\begin{align*}
\omega_1(u,v)&=u \left(1 - v\right),\\
\omega_2(u,v)&=a v \left(u - 1\right).\\
\end{align*}



### Double checking that our solutions satisfy the linearised symmetry condition

In [7]:
print("Now, we will plug in all our generators into the linearised symmetry conditions. What we want to find is that it only spits out the value 0.")
for index in range(len(generators_LV)):
    lin_sym_control,eq_sys_temp,monomials_temp = lin_sym_phase_plane(states_LV,omega_LV,list(generators_LV[index]))
    print("\t\tGenerator %d, value\t=\t%s"%(index+1,str(expand(lin_sym_control[0]))))

Now, we will plug in all our generators into the linearised symmetry conditions. What we want to find is that it only spits out the value 0.
		Generator 1, value	=	0
		Generator 2, value	=	0
		Generator 3, value	=	0
		Generator 4, value	=	0
		Generator 5, value	=	0
		Generator 6, value	=	0
		Generator 7, value	=	0
		Generator 8, value	=	0
		Generator 9, value	=	0
		Generator 10, value	=	0


Ok, so now we have printed all these steps 

# The BZ-model
Now we are considering the BZ-model:
\begin{equation}
  \begin{split}
    \dfrac{\mathrm{d}u}{\mathrm{d}\tau}&=\dfrac{1}{\varepsilon}v-\dfrac{1}{\varepsilon}\left(\dfrac{1}{3}u^3-u\right),\\
    \dfrac{\mathrm{d}v}{\mathrm{d}\tau}&=-u.\\    
    \end{split}
  \label{eq:BZ}
\end{equation}




In [8]:
# Dependent variables
u,v = symbols('u v')
# Define our parameter a in the LV model
e = symbols('e')
# Define the states
states_BZ = [u, v]
# Define our reaction terms
#omega_BZ = [(1/e)*v-(1/e)*(((1/3)*u**3)-u),-u]
omega_BZ = [(1/e)*v-(1/e)*((Rational(1,3)*u**3)-u),-u]
# Create the tangential ansatze
c_BZ, eta_BZ, M_BZ = create_tangential_ansatze(states_BZ,5)
# Plug in the tangential ansatze and calculate the resulting equations
lin_sym_BZ,eq_sys_BZ,monomials_BZ = lin_sym_phase_plane(states_BZ,omega_BZ,eta_BZ)
# Solve the system of equations for the unknown coefficients
solutions_BZ = solve(eq_sys_BZ,c_BZ,set=True)
# Extract the LHS and RHS 
LHS_BZ = solutions_BZ[0]
RHS_BZ = list(list(solutions_BZ[1])[0])
# Calculate and extract all generators
generators_BZ,trivial_generator = extract_generators(c_BZ,LHS_BZ,RHS_BZ,eta_BZ)
print("The calculated infinitesimal generators of the Lie group:")
print("\\begin{align*}")
for gen_index in range(len(generators_BZ)):
    if gen_index <len(generators_BZ)-1:
        print("X_{%d}&=%s\\partial_u+%s\\partial_v,\\\\"%(gen_index+1,latex(generators_BZ[gen_index][0]),latex(generators_BZ[gen_index][1])))
    else:
        print("X_{%d}&=%s\\partial_u+%s\\partial_v.\\\\"%(gen_index+1,latex(generators_BZ[gen_index][0]),latex(generators_BZ[gen_index][1])))
print("\\end{align*}")
if len(trivial_generator)>0:
    print("The trivial generator:")
    trivial_str = "\\begin{equation}\nX_{0}="+latex(trivial_generator[0][0])+"\\partial_u+"+latex(trivial_generator[0][1])+"\\partial_v.\n\\end{equation}"
    print("%s"%(trivial_str))
print("The reaction terms:")
print("\\begin{align*}\n\omega_1(u,v)&=%s,\\\\\n\omega_2(u,v)&=%s.\\\\\n\\end{align*}"%(latex(omega_BZ[0]),latex(omega_BZ[1])))

The calculated infinitesimal generators of the Lie group:
\begin{align*}
X_{1}&=\frac{u \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u^{2}\partial_v,\\
X_{2}&=\frac{v^{2} \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u v^{2}\partial_v,\\
X_{3}&=\frac{u v \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u^{2} v\partial_v,\\
X_{4}&=\frac{u^{3} - 3 u - 3 v}{3 e}\partial_u+u\partial_v,\\
X_{5}&=\frac{v \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u v\partial_v,\\
X_{6}&=\frac{u^{2} \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u^{3}\partial_v.\\
\end{align*}
The reaction terms:
\begin{align*}
\omega_1(u,v)&=\frac{v}{e} - \frac{\frac{u^{3}}{3} - u}{e},\\
\omega_2(u,v)&=- u.\\
\end{align*}


The calculated infinitesimal generators of the Lie group:
\begin{align*}
X_{1}&=\frac{u \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u^{2}\partial_v,\\
X_{2}&=\frac{v^{2} \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u v^{2}\partial_v,\\
X_{3}&=\frac{u v \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u^{2} v\partial_v,\\
X_{4}&=\frac{u^{3} - 3 u - 3 v}{3 e}\partial_u+u\partial_v,\\
X_{5}&=\frac{v \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u v\partial_v,\\
X_{6}&=\frac{u^{2} \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u^{3}\partial_v.\\
\end{align*}
The reaction terms:
\begin{align*}
\omega_1(u,v)&=\frac{v}{e} - \frac{\frac{u^{3}}{3} - u}{e},\\
\omega_2(u,v)&=- u.\\
\end{align*}


# The Brusselator
Now we are considering the Brusselator model:
\begin{equation}
  \begin{split}
    \dfrac{\mathrm{d}u}{\mathrm{d}\tau}&=1-(b-1)u+au^2 v,\\
    \dfrac{\mathrm{d}v}{\mathrm{d}\tau}&=bu-au^2 v.\\
    \end{split}
  \label{eq:Brusselator}
\end{equation}




In [9]:
# Dependent variables
u,v = symbols('u v')
# Define our parameter a in the LV model
a,b = symbols('a b')
# Define the states
states_Brusselator = [u, v]
# Define our reaction terms
omega_Brusselator = [1-((b-1)*u)+(a*(u**2)*v),b*u-(a*(u**2)*v)]
# Create the tangential ansatze
c_Brusselator, eta_Brusselator, M_Brusselator = create_tangential_ansatze(states_Brusselator,5)
# Plug in the tangential ansatze and calculate the resulting equations
lin_sym_Brusselator,eq_sys_Brusselator,monomials_Brusselator = lin_sym_phase_plane(states_Brusselator,omega_Brusselator,eta_Brusselator)
# Solve the system of equations for the unknown coefficients
solutions_Brusselator = solve(eq_sys_Brusselator,c_Brusselator,set=True)
# Extract the LHS and RHS 
LHS_Brusselator = solutions_Brusselator[0]
RHS_Brusselator = list(list(solutions_Brusselator[1])[0])
# Calculate and extract all generators
generators_Brusselator,trivial_generator = extract_generators(c_Brusselator,LHS_Brusselator,RHS_Brusselator,eta_Brusselator)
print("The calculated infinitesimal generators of the Lie group:")
print("\\begin{align*}")
for gen_index in range(len(generators_Brusselator)):
    if gen_index <len(generators_Brusselator)-1:
        print("X_{%d}&=%s\\partial_u+%s\\partial_v,\\\\"%(gen_index+1,latex(generators_Brusselator[gen_index][0]),latex(generators_Brusselator[gen_index][1])))
    else:
        print("X_{%d}&=%s\\partial_u+%s\\partial_v.\\\\"%(gen_index+1,latex(generators_Brusselator[gen_index][0]),latex(generators_Brusselator[gen_index][1])))
print("\\end{align*}")
if len(trivial_generator)>0:
    print("The trivial generator:")
    trivial_str = "\\begin{equation}\nX_{0}="+latex(trivial_generator[0][0])+"\\partial_u+"+latex(trivial_generator[0][1])+"\\partial_v.\n\\end{equation}"
    print("%s"%(trivial_str))
print("The reaction terms:")
print("\\begin{align*}\n\omega_1(u,v)&=%s,\\\\\n\omega_2(u,v)&=%s.\\\\\n\\end{align*}"%(latex(omega_Brusselator[0]),latex(omega_Brusselator[1])))

The calculated infinitesimal generators of the Lie group:
\begin{align*}
X_{1}&=- \frac{u \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u^{2} \left(a u v - b\right)}{a}\partial_v,\\
X_{2}&=- \frac{v \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u v \left(a u v - b\right)}{a}\partial_v,\\
X_{3}&=- \frac{u^{2} \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u^{3} \left(a u v - b\right)}{a}\partial_v,\\
X_{4}&=- \frac{v^{2} \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u v^{2} \left(a u v - b\right)}{a}\partial_v,\\
X_{5}&=- \frac{u v \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u^{2} v \left(a u v - b\right)}{a}\partial_v.\\
\end{align*}
The reaction terms:
\begin{align*}
\omega_1(u,v)&=a u^{2} v - u \left(b - 1\right) + 1,\\
\omega_2(u,v)&=- a u^{2} v + b u.\\
\end{align*}


The calculated infinitesimal generators of the Lie group:
\begin{align*}
X_{1}&=- \frac{u \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u^{2} \left(a u v - b\right)}{a}\partial_v,\\
X_{2}&=- \frac{v \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u v \left(a u v - b\right)}{a}\partial_v,\\
X_{3}&=- \frac{u^{2} \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u^{3} \left(a u v - b\right)}{a}\partial_v,\\
X_{4}&=- \frac{v^{2} \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u v^{2} \left(a u v - b\right)}{a}\partial_v,\\
X_{5}&=- \frac{u v \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u^{2} v \left(a u v - b\right)}{a}\partial_v.\\
\end{align*}
The reaction terms:
\begin{align*}
\omega_1(u,v)&=a u^{2} v - u \left(b - 1\right) + 1,\\
\omega_2(u,v)&=- a u^{2} v + b u.\\
\end{align*}


# Conclusion
I am not sure if all these generators are trivial, but let's present them and see what we think about it.